In [2]:
import numpy as np

In [3]:
corpus = np.load('corpus.npy')

In [10]:
k = 10
objective = list()
centers = corpus[np.random.choice(np.arange(corpus.shape[0]), k, replace=False)]
groups = np.array([np.argmin([np.linalg.norm(center - x) for center in centers]) for x in corpus])
objective.append(
    np.array([np.linalg.norm(corpus[i] - centers[groups[i]]) for i in range(corpus.shape[0])]).sum()
)

i = 0
while True:
    centers = np.array([corpus[groups == g].mean(axis=0) for g in range(k)])
    groups = np.array([np.argmin([np.linalg.norm(center - x) for center in centers]) for x in corpus])
    objective.append(
        np.array([np.linalg.norm(corpus[i] - centers[groups[i]]) for i in range(corpus.shape[0])]).mean()
    )
    if i > 2 and ((objective[-1] - objective[-2]) + (objective[-2] - objective[-3])) < 1.0:
        break
    i += 1

In [11]:
objective[-1]

np.float64(26.02381002185839)

In [21]:
corpus[groups == 0][0]

array([0, 0, 0, ..., 0, 0, 0])

How many articles in each cluster

In [27]:
{g: corpus[groups == g].shape[0] for g in range(10)}

{0: 540, 1: 125, 2: 1250, 3: 357, 4: 88, 5: 170, 6: 53, 7: 144, 8: 65, 9: 212}

Identify the words present in each cluster

In [24]:
words = np.load('words.npy')

In [42]:
cluster_word_histogram = corpus[groups == 5].sum(axis=0)

In [43]:
non_zero_i = np.where(cluster_word_histogram > 0)

In [44]:
cluster_hist_norm = cluster_word_histogram / np.linalg.norm(cluster_word_histogram)

In [48]:
cluster_hist_norm.dtype

dtype('float64')

In [45]:
print(*words[non_zero_i])

boy strike city picture body middle rubble blue dust debris child morning missile ministry border photo grandmother brother state administration chief evening address hospital result invasion severity attack day village region store cafe sense war support half tank fleet start information monitoring group source intelligence website evidence equipment week toll analyst chance focus territory battlefield fight way count month technology battle enemy armor design round ammunition blast box effect storage front reserve paper lot action force replacement amount term figure infantry look terror total magazine suburb court goal religion publication death title newspaper decision bomb descent police affiliate time face drone training bombing prison investigation plot phone part shooting woman airport flight man return nation security course prosecutor wife terrorism terrorist network guest maintenance worker officer building street range name search area northeast food gas station law enforce

In [100]:
values_i = np.argsort(cluster_hist_norm)[::-1]

In [102]:
words[values_i]

array(['ve', 'rain', 'hat', ..., 'duke', 'slalom', 'pope'], dtype='<U16')